In [ ]:
# default_exp core
%load_ext lab_black

In [ ]:
# hide
from nbdev.showdoc import *

In [ ]:
# export

import os
import sys
import argparse
import pandas as pd
from pathlib import Path
from collections import namedtuple
from subprocess import Popen, PIPE

python = sys.executable
home = Path(os.path.expanduser("~"))
blogslicer = Path(home / Path("github/blogslicer/blogslicer/core.py"))
if hasattr(__builtins__, "__IPYTHON__") or __name__ != "__main__":
    from IPython.display import display, Markdown

    h1 = lambda text: display(Markdown(f"# {text}"))
    h2 = lambda text: display(Markdown(f"## {text}"))
    h3 = lambda text: display(Markdown(f"### {text}"))

    file = "sites.csv"
else:
    h1 = lambda text: print(f"# {text}")
    h2 = lambda text: print(f"## {text}")
    h3 = lambda text: print(f"## {text}")

    aparser = argparse.ArgumentParser()
    add_arg = aparser.add_argument
    add_arg("-f", "--file", required=True)
    args = aparser.parse_args()
    file = args.file

h1("Generaring sites...")
file_obj = Path(file)
df = pd.read_csv(file_obj, delimiter="|")
df = df.applymap(lambda x: x.strip())
df.columns = [x.strip() for x in df.columns]
Site = namedtuple("Site", "path, apex, title, gaid, tagline")

h2(f"Python: {python}")
h2(f"Blogslicer: {blogslicer}")
print()
for index, series in df.iterrows():
    site = Site(**series.to_dict())
    h3(site.apex)
    here = Path(home / site.path)
    cmd = f'{python} {blogslicer} -p {here} -t "{site.title}" -s "/blog/" -a "Mike Levin"'
    print(cmd)
    print()
    with Popen(args=cmd, cwd=here, stdout=PIPE, stderr=PIPE, shell=True) as pout:
        err = pout.stderr.read()
        if len(err) > 0:
            h3(err)
            break
        for line in pout.stdout.readlines():
            print(line.decode().strip())
print('Done')

# Generaring sites...

## Python: C:\Users\mikle\AppData\Roaming\jupyterlab-desktop\jlab_server\python.exe

## Blogslicer: C:\Users\mikle\github\blogslicer\blogslicer\core.py

### GuerillaTech.com

C:\Users\mikle\AppData\Roaming\jupyterlab-desktop\jlab_server\python.exe C:\Users\mikle\github\blogslicer\blogslicer\core.py -p C:\Users\mikle\github\GuerillaTech.com -t "Guerilla Tech" -s "/blog" -a "Mike Levin"

## FILE: C:\Users\mikle\github\GuerillaTech.com/_posts/2022-04-22-post-1.md


...

C:\Users\mikle\AppData\Roaming\jupyterlab-desktop\jlab_server\python.exe C:\Users\mikle\github\blogslicer\blogslicer\core.py -p C:\Users\mikle\github\Mike-Levin.com -t "Mike Levin Dot Com" -s "/blog" -a "Mike Levin"

## FILE: C:\Users\mikle\github\Mike-Levin.com/_posts/2022-04-22-post-1.md


### MikeAtEleven.com

C:\Users\mikle\AppData\Roaming\jupyterlab-desktop\jlab_server\python.exe C:\Users\mikle\github\blogslicer\blogslicer\core.py -p C:\Users\mikle\github\MikeAtEleven.com -t "Mike At Eleven" -s "/blog" -a "Mike Levin"

## FILE: C:\Users\mikle\github\MikeAtEleven.com/_posts/2022-06-05-post-23.md
## FILE: C:\Users\mikle\github\MikeAtEleven.com/_posts/2022-05-07-post-22.md
## FILE: C:\Users\mikle\github\MikeAtEleven.com/_posts/2022-05-06-post-21.md
## FILE: C:\Users\mikle\github\MikeAtEleven.com/_posts/2022-05-05-post-20.md
## FILE: C:\Users\mikle\github\MikeAtEleven.com/_posts/2022-05-04-post-19.md
## FILE: C:\Users\mikle\github\MikeAtEleven.com/_posts/2022-05-04-post-18.md
## FILE: C:\Users\mikle\github\MikeAtEleven.com/_posts/2022-05-04-post-17.md
## FILE: C:\Users\mikle\github\MikeAtEleven.com/_posts/2022-05-04-post-16.md
## FILE: C:\Users\mikle\github\MikeAtEleven.com/_posts/2022-05-04-post-15.md
## FILE: C:\Users\mikle\github\MikeAtEleven.com/_posts/2022-05-04-post-14.md
## FILE: C:\Use

### MikeLev.in

C:\Users\mikle\AppData\Roaming\jupyterlab-desktop\jlab_server\python.exe C:\Users\mikle\github\blogslicer\blogslicer\core.py -p C:\Users\mikle\github\MikeLev.in -t "Mike Levin" -s "/blog" -a "Mike Levin"



In [ ]:
!nbdev_clean_nbs
!nbdev_build_lib

Converted 00_core.ipynb.
Converted index.ipynb.
